# 5. Tools

Must read:
- [Tool calling](https://python.langchain.com/docs/concepts/tool_calling/)
- [How to pass tool outputs to chat models](https://python.langchain.com/docs/how_to/tool_results_pass_to_model/)

Tool calling, also known as function calling, enables AI models to interact with systems like APIs or databases by responding in a schema-specific format. It involves four key steps: tool creation (using the @tool decorator to define a tool with its schema), tool binding (connecting the tool to a model via .bind_tools()), tool calling (where the model determines when to use the tool based on input relevance), and tool execution (where the tool runs using model-provided arguments). 

<img src="https://python.langchain.com/assets/images/tool_calling_concept-552a73031228ff9144c7d59f26dedbbf.png" height="200">

LangChain provides a standardized interface for integrating tools with models, supporting workflows where the model invokes tools only when necessary, ensuring responses conform to the tool’s schema. Best practices include using simple, well-named, narrowly scoped tools to enhance model performance and usability.

1) **Tool Creation**: Use the @tool decorator to create a tool. A tool is an association between a function and its schema. 
2) **Tool Binding**: The tool needs to be connected to a model that supports tool calling. 

In [126]:
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage
from langchain_ollama import ChatOllama
from langchain_groq import ChatGroq
import os

In [127]:
# Initialize the language model with specific settings
llm = ChatOllama(
    base_url=os.getenv("OLLAMA_SERVER"),
    model = "mistral",
    temperature = 0.8
)

In [128]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiply a and b."""
    return a * b

In [ ]:
# without tools you would get a simple LLM answer
result = llm.invoke("who are you?")
print(f"Simple LLM result: {result.content}")

In [130]:
# Now, if you want to use the tool, you need to "bind" it to the LLM
llm_with_tools = llm.bind_tools(tools=[multiply])

In [131]:
# After that, you can use the tool in your queries
query    = "What is 2 multiplied by 3?"
messages = [HumanMessage(content=query)]
result   = llm_with_tools.invoke(messages)

In [ ]:
# Method one: execute the tool call directly
# The result will contain the tool call
for tool_call in getattr(result, "tool_calls", []):
    if tool_call["name"] == "multiply":
        args = tool_call["args"]
        tool_result = multiply.run(args)  # Pass arguments as a dictionary
        print(f"Tool Execution Result: {args} => {tool_result}")

In [ ]:
# Method two: execute the tool call using the LLM
# Using .invoke() is a valid and powerful approach when handling tools, 
# as it ensures that the tool execution is consistent with LangChain’s 
# interface and allows for additional flexibility (e.g., returning AI messages). 

messages = []  # This will store the sequence of interactions
# Process each tool call in the response
for tool_call in getattr(result, "tool_calls", []):
    # Map the tool name to the corresponding tool function
    selected_tool = {"multiply": multiply}[tool_call["name"].lower()]
    # Execute the tool using `.invoke()` and add the response to messages
    tool_msg = selected_tool.invoke(tool_call["args"])
    messages.append(tool_msg)

# Print the updated messages list
for msg in messages:
    print(f"Tool Response: {msg}")

The main difference is output format and intended use:

1.	`.invoke()`: Returns a structured result (like AIMessage) aligned with LangChain’s schema, making it suitable for workflows requiring conversational context or chaining multiple tools.
2.	`.run()`: Directly executes the tool and returns a raw result (e.g., 6), which is simpler and better for standalone use.

Use `.invoke()` for structured workflows; use `.run()` for quick, direct execution.